In [1]:
using Pkg
Pkg.add("MLDatasets")
Pkg.add("Flux")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [2]:
using MLDatasets
train_data = MLDatasets.MNIST(split=:train)
test_data  = MLDatasets.MNIST(split=:test)

dataset MNIST:
  metadata  =>    Dict{String, Any} with 3 entries
  split     =>    :test
  features  =>    28×28×10000 Array{Float32, 3}
  targets   =>    10000-element Vector{Int64}

In [3]:
using Flux
function loader(data; batchsize::Int=1)
    x1dim = reshape(data.features, 28 * 28, :) # reshape 28×28 pixels into a vector of pixels
    yhot  = Flux.onehotbatch(data.targets, 0:9) # make a 10×60000 OneHotMatrix
    Flux.DataLoader((x1dim, yhot); batchsize, shuffle=true)
end

loader (generic function with 1 method)

In [4]:
train_loader = loader(train_data)
test_loader = loader(test_data)

10000-element DataLoader(::Tuple{Matrix{Float32}, OneHotArrays.OneHotMatrix{UInt32, Vector{UInt32}}}, shuffle=true)
  with first element:
  (784×1 Matrix{Float32}, 10×1 OneHotMatrix(::Vector{UInt32}) with eltype Bool,)

In [5]:
INPUT_SIZE = 196
HIDDEN_SIZE = 64
OUTPUT_SIZE = 10 

10

In [41]:
hidden_activation = tanh
output_activation = identity

identity (generic function with 1 method)

In [87]:
input_to_hidden_weights = randn(Float32, INPUT_SIZE, HIDDEN_SIZE)
hidden_to_hidden_weights = randn(Float32, HIDDEN_SIZE, HIDDEN_SIZE)
hidden_to_output_weights = randn(Float32, HIDDEN_SIZE, OUTPUT_SIZE)

hidden_bias = randn(Float32, 1, HIDDEN_SIZE)
output_bias = randn(Float32, 1, OUTPUT_SIZE)

1×10 Matrix{Float32}:
 0.585381  -1.3694  1.72047  -0.455503  …  1.49037  0.318298  0.132169

In [47]:
for (sample, label) in test_loader
    s1 = hidden_activation.(transpose(sample[1:196]) * input_to_hidden_weights)
    o1 = s1 * hidden_to_output_weights
    
    s2 = hidden_activation.(transpose(sample[197:392]) * input_to_hidden_weights + s1)
    o2 = s2 * hidden_to_output_weights
    
    s3 = hidden_activation.(transpose(sample[393:588]) * input_to_hidden_weights + s2)
    o3 = s3 * hidden_to_output_weights
    
    s4 = hidden_activation.(transpose(sample[589:784]) * input_to_hidden_weights + s3)
    o4 = s4 * hidden_to_output_weights
end

In [114]:
prev_hidden = transpose(zeros(HIDDEN_SIZE))

for (sample, label) in test_loader
    outputs = []
    for i in range(1, step=196, stop=784)
        lb = i
        ub = i+195

        x = transpose(sample[lb:ub])
        xi = x * input_to_hidden_weights
        xh = hidden_activation.(
            xi + prev_hidden * hidden_to_hidden_weights + hidden_bias
        )
        prev_hidden = xh
        xo = xh * hidden_to_output_weights + output_bias
        push!(outputs, xo)
    end
    # println(last(outputs))
    # println(label)
    # println()
end